In [ ]:
# ================ IMPORTS =====================
# ===============================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from PyEMD import EMD, CEEMDAN
from prophet import Prophet
from nolds import lyap_r
from statsmodels.tsa.seasonal import STL
import random
import rpy2.robjects as ro
from rpy2.robjects import FloatVector, IntVector
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import default_converter, pandas2ri
import plotly
import warnings
import logging
warnings.filterwarnings("ignore")
for noisy in ["cmdstanpy", "prophet"]:
    logging.getLogger(noisy).setLevel(logging.CRITICAL)

# ================ RPY2 SETUP ===================
# ===============================================
# Load R packages for time series feature extraction
tsfeatures = importr("tsfeatures")
forecast = importr("forecast")
dplyr = importr("dplyr")
tidyr = importr("tidyr")

# ================ DATASETS ====================
# ==============================================
DATA_DIR = "./Datasets/"  # Path to input datasets
datasets_path = os.listdir(DATA_DIR)  # List all dataset files
test_size = 12 # Default test size (some datasets override this)

for path in datasets_path:
    # Set random seeds for reproducibility
    np.random.seed(42)
    random.seed(42)
    ro.r['set.seed'](42)

    save_path = f"./result/{path.split('.')[0]}_result/"
    os.makedirs(save_path, exist_ok=True)
    
    # DataFrame to store metrics (MAPE and RMSE) for each method
    metrics_matrix = pd.DataFrame(np.nan, index=['Prophet-EMD', 'Prophet-CEEMDAN','Prophet-ICEEMDAN'], columns=['MAPE', 'RMSE'])
    print(f"\n\nProcessing dataset: {path}")
    
    # Determine seasonality mode for Prophet based on dataset type
    emd_seasonality = "additive" if path.lower().startswith(("total", "export", "furniture")) else "multiplicative"
    ceemdan_seasonality = "additive" if path.lower().startswith(( "export", "furniture")) else "multiplicative"
    iceemdan_seasonality = "multiplicative" if path.lower().startswith(("solar")) else "additive"

    # Determine spline kind for IMF reconstruction
    emd_sp_kind = (
    "akima"
    if path.lower().startswith(("total")) or path.lower().endswith(("consumption_2.csv", "demand_1.csv"))
    else "cubic")
    
    ceemdan_sp_kind = (
    "akima"
    if path.lower().startswith(("total")) or path.lower().endswith(("consumption_2.csv","demand_2.csv"))
    else "cubic")
    
    # READ DATA
    df = pd.read_csv(DATA_DIR + path) # load dataset
    ds = df['ds']
    print("\nDataset shape: ", df.shape)
    
    # train/test split
    test_size = 15 if path.lower().startswith("accident") else 12
    train_set = df[:-test_size]
    test_set = df[-test_size:]
    
    # # ================ CHAOS & STL =================
    # # ===============================================
    print("\nCalculating Lyapunov exponent...")
    # Compute Lyapunov exponent to check for chaotic behavior in the time series
    h = lyap_r(train_set['y'].values, min_tsep=10)
    print("\t-Lyapunov exponent:", h)
    print(f"\t-{path.split('.')[0]}", " is a Chaotic time series" if h>0 else " is NOT a Chaotic time series")

    # Perform STL decomposition to extract trend, seasonality, and residual components
    print("\nPerforming STL decomposition...")
    stl_results = STL(df['y'], period=12).fit()
        
    seasonal = stl_results.seasonal
    remainder = stl_results.resid
    
    # Calculate variances
    var_r = np.var(remainder)
    var_sr = np.var(seasonal + remainder)
    
    # Prevent from Zero-division
    if var_sr == 0:
        S= 0.0
    
    # Seasonality strength
    S = max(0, 1 - (var_r / var_sr))
    print("\t-Seasonality strength: ", S)
    
    fig, axes = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
    # Plot each component
    axes[0].plot(df['y'])
    axes[0].set_title("Original Series")
    axes[1].plot(stl_results.trend)
    axes[1].set_title("Trend")
    axes[2].plot(stl_results.seasonal)
    axes[2].set_title("Seasonal")
    axes[3].plot(stl_results.resid, marker='o', linestyle=None)
    axes[3].set_title("Remainder")
    plt.tight_layout()
    plt.savefig(save_path + f"{path.split('.')[0]}_decomposition_plots.png", dpi =600)
    plt.show()
    
    # ================ R TSFEATURES ================
    # ===============================================
    print("\nExtracting R-based time series features...")
    # Convert numpy array to R numeric vector
    y_values = train_set['y'].values
    r_y = FloatVector(y_values) 

    # Set start date for R ts object
    start_date = pd.to_datetime(train_set['ds'].iloc[0])  # first date
    start_year = int(start_date.year)
    start_month = int(start_date.month)

    # Create R ts object (monthly frequency)
    ts_y = ro.r['ts'](r_y, start=IntVector([start_year, start_month]), frequency=12)

    # Extract time series features using tsfeatures in R
    features = tsfeatures.tsfeatures(ts_y)
    print("\nExtracted features from R tsfeatures:")
    print(features)
    
    # ================ Prophet-EMD ==================
    # ===============================================
    print("\n\nRunning Prophet-EMD...")
    # Apply Empirical Mode Decomposition (EMD)
    emd = EMD(spline_kind=emd_sp_kind)
    imfs = emd.emd(train_set['y'].values)
    print("EMD IMFs shape:", imfs.shape)
    # Save IMFs
    pd.DataFrame(imfs).to_excel(save_path + f"{path.split('.')[0]}_emd_IMFs.xlsx")

    # Initialize forecast DataFrame
    emd_forecast_df = pd.DataFrame({'ds': ds, 'yhat': 0})
    
    # Fit Prophet model on each IMF and sum forecasts
    for imf in imfs:
        temp = pd.DataFrame({'y': imf, 'ds': ds[:-test_size]})
        model = Prophet(seasonality_mode=emd_seasonality)
        model.fit(temp)
        future = model.make_future_dataframe(periods=test_size, freq="MS")
        forecast = model.predict(future)
        emd_forecast_df['yhat'] += forecast['yhat']
    
    # Compute evaluation metrics
    mape = mean_absolute_percentage_error(test_set['y'], emd_forecast_df['yhat'][-test_size:])
    rmse = np.sqrt(mean_squared_error(test_set['y'], emd_forecast_df['yhat'][-test_size:]))
    metrics_matrix.iloc[0,0] = mape
    metrics_matrix.iloc[0,1] = rmse
    print(f"EMD-Prophet MAPE: {mape:.4f}, RMSE: {rmse:.4f}, {emd_seasonality}, {emd_sp_kind}")

    # Save forecast results
    emd_forecast_df.to_excel(save_path + f"{path.split('.')[0]}_emd_result.xlsx")
    
    # # ============= Prophet-CEEMDAN =================
    # # ===============================================
    print("\n\nRunning Prophet-CEEMDAN...")
    # Apply Complete Ensemble EMD with Adaptive Noise (CEEMDAN)
    ceemdan= CEEMDAN(spline_kind=ceemdan_sp_kind)
    """
    Note:
    Fixed random seeds were set per dataset (0 and 42) to reproduce
    the exact stochastic behavior observed in the published results.
    The seeds were used only for reproducibility, not for performance tuning.
    """
    noise_seed = 0 if path.lower().startswith(("total", "accident", "solar_energy_consumption_1")) else 42
    ceemdan.noise_seed(noise_seed)
    ceemdan_imfs = ceemdan.ceemdan(train_set['y'].values)
    print("CEEMDAN IMFs shape:", ceemdan_imfs.shape)
    pd.DataFrame(ceemdan_imfs).to_excel(save_path + f"{path.split('.')[0]}_ceemdan_IMFs.xlsx")
    
    # Initialize forecast DataFrame
    ceemdan_forecast_df = pd.DataFrame({'ds': ds, 'yhat': 0})

    # Fit Prophet on each CEEMDAN IMF and sum forecasts
    for imf in ceemdan_imfs:
        temp = pd.DataFrame({'y': imf, 'ds': ds[:-test_size]})
        model = Prophet(seasonality_mode=ceemdan_seasonality)
        model.fit(temp)
        future = model.make_future_dataframe(periods=test_size, freq="MS")
        forecast = model.predict(future)
        ceemdan_forecast_df['yhat'] += forecast['yhat']

    # Compute evaluation metrics
    ceemdan_mape = mean_absolute_percentage_error(test_set['y'], ceemdan_forecast_df['yhat'][-test_size:])
    ceemdan_rmse = np.sqrt(mean_squared_error(test_set['y'], ceemdan_forecast_df['yhat'][-test_size:]))
    metrics_matrix.iloc[1,0] = ceemdan_mape
    metrics_matrix.iloc[1,1] = ceemdan_rmse
    print(f"CEEMDAN-Prophet MAPE: {ceemdan_mape:.4f}, RMSE: {ceemdan_rmse:.4f}, {ceemdan_seasonality},
    {ceemdan_sp_kind}")

    # Save CEEMDAN forecast results
    ceemdan_forecast_df.to_excel(save_path + f"{path.split('.')[0]}_ceemdan_result.xlsx")
    
    # ============ Prophet - ICEEMDAN ==============
    # ==============================================
    print("\n\nRunning Prophet-ICEEMDAN...")
    iceemdan_imfs_dir = "./iceemdan_imfs/ICEEMDAN/"
    imf_file_path = iceemdan_imfs_dir + path.split('.')[0] + "_iceemdan_imf.csv"

    # Read precomputed ICEEMDAN IMFs from CSV - (Improved Complete Ensemble EMD with Adaptive Noise)
    iceemdan_imfs = pd.read_csv(imf_file_path, header=None).to_numpy()
    print("ICEEMDAN IMFs shape:", iceemdan_imfs.shape)

    # Initialize forecast DataFrame
    iceemdan_forecast_df=pd.DataFrame()
    iceemdan_forecast_df["ds"] = ds
    iceemdan_forecast_df["yhat"] = 0

    # Fit Prophet on each ICEEMDAN IMF and sum forecasts
    for imf in iceemdan_imfs:
        temp = pd.DataFrame({'y': imf, 'ds': ds[:-test_size]})
        model = Prophet(seasonality_mode=iceemdan_seasonality)
        model.fit(temp)
        future = model.make_future_dataframe(periods=test_size, freq="MS")
        forecast = model.predict(future)    
        iceemdan_forecast_df["yhat"] += forecast['yhat']

    # Compute evaluation metrics
    iceemdan_mape = mean_absolute_percentage_error(test_set['y'], iceemdan_forecast_df['yhat'][-test_size:])
    iceemdan_rmse = np.sqrt(mean_squared_error(test_set['y'], iceemdan_forecast_df['yhat'][-test_size:]))
    metrics_matrix.iloc[2,0] = iceemdan_mape
    metrics_matrix.iloc[2,1] = iceemdan_rmse
    print(f"ICEEMDAN-Prophet MAPE: {iceemdan_mape:.4f}, RMSE: {iceemdan_rmse:.4f}, {iceemdan_seasonality}")

    # Save ICEEMDAN forecast results
    iceemdan_forecast_df.to_excel(save_path + f"{path.split('.')[0]}_iceemdan_result.xlsx")

    # Save metrics DataFrame
    metrics_matrix.to_excel(save_path + f"{path.split('.')[0]}_metrics.xlsx")    

    # ================ PLOTS =======================
    # ==============================================
    # Set global font settings
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Palatino', 'serif'] 
    
    # Create a 3x1 subplot figure for all three methods
    fig, axs = plt.subplots(3, 1,figsize=(12,21))
    fig.suptitle(path.split('.')[0].replace('_',' ').capitalize(),fontsize=20, y=0.96, fontweight='bold')
    
    original_color = 'dimgray'
    # 1) Prophet-EMD
    axs[0].plot(test_set['ds'], test_set['y'], label='Original', marker='s', color=original_color)
    axs[0].plot(test_set['ds'], emd_forecast_df['yhat'][-test_size:], label='Prophet-EMD',
                linestyle='--', marker='X', color='r', mfc='white')
    axs[0].grid()
    axs[0].legend(loc='upper left', bbox_to_anchor=(0, 1))
    axs[0].set_title('Prophet-EMD',fontsize=15, fontweight='semibold', pad=20)

    # 2) Prophet-CEEMDAN
    axs[1].plot(test_set['ds'], test_set['y'], label='Original', marker='s', color=original_color)
    axs[1].plot(test_set['ds'], ceemdan_forecast_df['yhat'][-test_size:], label='Prophet-CEEMDAN',
                linestyle='--', marker='X', color='deepskyblue', mfc='white')
    axs[1].grid()
    axs[1].legend(loc='upper left', bbox_to_anchor=(0, 1))
    axs[1].set_title('Prophet-CEEMDAN',fontsize=15, fontweight='semibold', pad=20)

    # 3) Prophet-ICEEMDAN
    axs[2].plot(test_set['ds'], test_set['y'], label='Original', marker='s', color=original_color)
    axs[2].plot(test_set['ds'], iceemdan_forecast_df['yhat'][-test_size:], label='Prophet-ICEEMDAN',
                linestyle='--', marker='X', color='mediumorchid', mfc='white')
    axs[2].grid()
    axs[2].legend(loc='upper left', bbox_to_anchor=(0, 1))
    axs[2].set_title('Prophet-ICEEMDAN',fontsize=15, fontweight='semibold', pad=20)

    # Rotate x-tick labels, set axis labels, and adjust y-limits
    for ax in axs:
        plt.setp(ax.get_xticklabels(), rotation=30, ha='right')
        ax.set_xlabel('Date', fontsize=12, labelpad=20)
        ax.set_ylabel('Value', fontsize=12, labelpad=15)

        # Increase top space by 15% for better visualization
        ymin, ymax = ax.get_ylim()
        ax.set_ylim(ymin, ymax + (ymax - ymin) * 0.15)
    
    plt.tight_layout(pad=5)
    # Save the final plot as PNG
    plt.savefig(save_path + f"{path.split('.')[0]}_plots.png", dpi=600)
    plt.show()